![navis logo](https://github.com/navis-org/navis/raw/master/docs/_static/logo_new_banner.png)

# NeuroPython Workshop 2024 - <span style="color:rgb(250,175,3)">NAVis</span> Exercise 1

_Author: Philipp Schlegel (University of Cambridge)_

_Date: September 23, 2024_

_Source: https://github.com/navis-org/neuropython2024/exercises/navis/navis_exercise_1.ipynb_

In this first exercise we will introduce the basic concepts of <span style="color:rgb(250,175,3)">NAVis</span>. You will learn about:

1. Different neuron types 
2. Converting between neuron meshes and skeletons
3. Neuron properties
4. Lists of neurons 
5. Ways to plot neurons

### Requirements 

For this exercise, you will need a full, up-to-date install of <span style="color:rgb(250,175,3)">NAVis</span>:

```shell 
pip install "navis[all]" -U
```
### Part I: Neuron Types & Properties

<span style="color:rgb(250,175,3)">NAVis</span> supports 4 different types of neuron representations:
- `navis.MeshNeuron` = meshes
- `navis.TreeNeuron` = skeletons 
- `navis.Dotprops` = dotprops 
- `navis.VoxelNeuron` = images

For this tutorial, we will focus in skeletons and meshes as these are the types you will most likely work with.

#### Skeletons

In [ ]:
import navis

# This should be 1.8.0 or higher
print(navis.__version__)

In [ ]:
# Load one of the example skeletons
n = navis.example_neurons(1, kind="skeleton")

# Show a short summary of the skeleton
n

For skeletons, the underlying data is the SWC node table which you can access via the `.nodes` property:

In [ ]:
# The nodes table is a simple pandas DataFrame
n.nodes.head()

Properties such as the ones seen in the summary can be accessed similarly:

In [ ]:
n.cable_length

Try some other parameter - e.g. `n.n_leafs`!

You may have noticed that our example neuron has a `units` property of `8 nanometer`. 

In [ ]:
n.units

If you know your neurons units, it's useful to set that property because it allows us to do stuff like this:

In [ ]:
n.cable_length * n.units

Units can be set manually using a string:

In [7]:
n.units = "8 nm"

#### Somata

You've probably already seen the `.soma` property on our example neuron:

In [ ]:
n.soma

For skeletons, `.soma` is the ID of the node that corresponds to the neuron's soma. 

<span style="color:rgb(250,175,3)">NAVis</span> tries its best to infer the soma from whatever data you throw at it. For example when reading SWC files, it will look for a `label` column and failing that will look for nodes with a large radius. You can also set the soma manually:

In [9]:
n.soma = 4177

The `.soma` property is used for example for plotting and a few other functions, so it's always good to check this is set correctly.

##### Roots

Skeletons are directed acyclic graphs (= trees) where each node has exactly one parent. The only exception is the neuron's "root", the one node that does not have a parent:

In [ ]:
# Like `.soma`, these are node IDs
n.root

Our example neuron is fully connected and so it only has a single root. If there are any breaks in the structure, you would find multiple root nodes.

Why should we care about the skeleton's root? Roots are important because they are used as points of reference.
For example, when calculating Strahler Indices (SI) we walk from the leafs to the root and if that's not correctly set your SI won't make much sense. 

Currently, our neuron isn't actually rooted to its soma - let's fix that!

In [ ]:
# Reroot the skeleton at the soma
navis.reroot_skeleton(n, n.soma, inplace=True)

# Check that the new root is correct
n.root

Above function call also introduced an important concept: the `inplace` parameter. If you are used to `pandas` this should look familiar!
Many functions in <span style="color:rgb(250,175,3)">NAVis</span> accept this parameter:

- `inplace=False` (default): the neuron(s) are copied and those copies are modifid and returned 
- `inplace=True`: the original neurons are modified 

The latter is useful if you don't need to keep the originals - it saves time and memory by avoiding having to copy data. 

#### Meshes


In [ ]:
# Load one of the example neuron meshes
m = navis.example_neurons(1, kind="mesh")

# Show a short summary of the mesh
m

As you can see the summary for meshes (i.e. `MeshNeurons`) looks quite a bit different from those for skeletons (i.e. `TreeNeurons`).
That's because in contrast to skeletons, neuron meshes don't immediately give us a sense of topology to calculate leafs, branch points, cable length 
and so on. For that, we have to skeletonize them - which we'll demo below.

The basic data for meshes are vertices (as x/y/z positions in space) and faces (as vertices indices):

In [ ]:
m.vertices

In [ ]:
m.faces

<span style="color:rgb(250,175,3)">NAVis</span> tries to make it easy to move between representations. In particular the conversion _to_ skeletons is very useful
as it allows us to analyse the neuron's topology.

For `MeshNeurons` that's fairly straight forward:

In [ ]:
# Skeletonize the mesh
sk = navis.skeletonize(m)
sk

**Important**: while the conversion function try to use sensible defaults, you may still want to inspect the results carefully and make adjustements if necessary!

You may also notice that some functions such as e.g. `navis.strahler_index` accept `MeshNeurons` even though they probably operate on a skeleton:

In [ ]:
# Calculate the Strahler index for the mesh
navis.strahler_index(m)

# The per-vertex Strahler index is attached as a new property
m.strahler_index

The above works because <span style="color:rgb(250,175,3)">NAVis</span> (lazily) generates a skeleton, calculates the Strahler index on it and then maps the indices
back onto the mesh. The skeleton is then cached for future use:

In [ ]:
# The .skeleton property is generated lazily on first access
m.skeleton

We won't cover this here but you can adjust the skeleton-representation manually - e.g. by using `m.skeletonize()`!

### Lists of Neurons

So far, we've only looked at single neurons but in reality you will likely work with a whole bunch at a time.
For that <span style="color:rgb(250,175,3)">NAVis</span> has `NeuronLists`:

In [ ]:
# Ask for multiple skeletons
nl = navis.example_neurons(3, kind="skeleton")

# You can also manually construct NeuronLists from individual neurons like so:
# nl = navis.NeuronList([n1, n2, n3])

# Show a short summary of the list of skeletons
nl

`NeuronLists` behave similar to numpy arrays in that we can index them like so:

In [ ]:
# Get the first neuron in the list
nl[0]

In [ ]:
# Index by a slice
nl[1:3]

In [ ]:
# Index by a boolean array
nl[nl.cable_length > 300_000]

In [ ]:
# Iterate over the neurons in the list
for neuron in nl:
    print(neuron.id)

In addition to the list/array-like ways to index into the `NeuronList`, you can also index by ID similar to `.loc` in `pandas`:

In [ ]:
# Grab a single neuron by ID
nl.idx[1734350908]

In [ ]:
# Grab multiple neurons by ID
nl.idx[[1734350908, 722817260]]

So `NeuronLists` make it fairly easy to organize your neurons but what else are they good for?

They also make it easy to access your neurons properties:

In [ ]:
# Get the number of nodes in each neuron
nl.n_nodes

This access is generic and should work for any neuron property:

In [ ]:
nl.id

We can also use the `NeuronList` to add/modify neuron properties:

In [ ]:
# Set a "my_property" attribute on the neurons
nl.set_neuron_attributes(["a", "b", "c"], name="my_property")

nl[0].my_property

See also `NeuronList.add_meta_data()` for adding a whole bunch of properties at once.

### Part II: Plotting

<span style="color:rgb(250,175,3)">NAVis</span> has various functions for plotting neurons:

- `navis.plot3d`
- `navis.plot2d` 
- `navis.plot1d` 
- `navis.plot_flat`

Here, we will focus on `plot3d` and `plot2d` but check out the online tutorials to learn about the rest!


#### Plot 2D

`navis.plot2d` uses `matplotlib` to visualize neurons. While `matplotlib` has 3d axes which allow you to plot
objects at oblique angles, its capabilities to render multiple objects correctly layered are limited. It's 
also fairly slow for large scenes which means it's hard to use interactively. 

On the plus side, we get very nice, clean-looking figures and there are tons of ways of customizing them.

In [ ]:
# Plot a single neuron
fig, ax = navis.plot2d(
    nl[0],
    color="k",  # set the color
    radius=True,  # use the radius column to set the width of the lines
    view=("x", "-y"),  # set the view
    method="2d",  # this determines whether we use 2d or 3d matplotlib axes
)

In [ ]:
# Plot a bunch of neurons
fig, ax = navis.plot2d(
    nl,  # pass a list of neurons
    palette="tab10",
    radius=True,
    view=("x", "-y"),
    method="2d",
)

You may have noticed that the skeletons look fairly plastic - that's because by default
<span style="color:rgb(250,175,3)">NAVis</span> will use the radius (if present). That 
typically looks better at the cost of slower rendering. To switch off radii:

In [ ]:
# Plot a bunch of neurons
fig, ax = navis.plot2d(
    nl,
    palette="tab10",
    radius=False,  # do not use the radius
    linewidth=.5,  # set the width of the lines
    view=("x", "-y"),
    method="2d",
)

You can always change the initial camera position using the `view` parameter. If you use 3D axes via `method="3d"`,
you can also rotate the camera:

In [ ]:
# Plot a bunch of neurons
fig, ax = navis.plot2d(
    nl,
    palette="tab10",
    radius=False,
    linewidth=1,
    view=("x", "-y"),
    method="3d",  # set method to 3d - try out `3d_complex`!
    non_view_axes3d=True  # show the non-view axes
)

# Move the camera around
ax.elev = -20
ax.azim = 45
ax.roll = 180

#### Plot 3D 

Interactive 3D plots can be generated with `navis.plot3d`. Depending on what's installed on your system and whether you are in a Jupyter environment or in e.g. a terminal, <span style="color:rgb(250,175,3)">NAVis</span> will use different backends:

- `plotly` is the default backend for Jupyter 
- `octarine3d` will be used by default in the terminal or scripts 
- `vispy` is the fallback for `octarine` 

In [ ]:
# This should produce a plotly plot
fig = navis.plot3d(nl, palette='tab10')

In [ ]:
# This should produce an octarine plot (if that backend is installed)
# Importantly: this will NOT work in Google Colab
viewer = navis.plot3d(nl, palette='tab10', backend='octarine')

Unlike `plotly` figures which are static once generated, `octarine` (and `vispy`) viewers can be manipulated further.

Run these cells one-by-one and look at the viewer above:

In [34]:
# Set all neurons to red
viewer.set_colors('r')

In [35]:
# Clear the viewer
viewer.clear()

In [36]:
# Add a single neuron
viewer.add(nl[0])

In [37]:
# Change camera position
viewer.set_view("XY")  # this also accepts more complex, custom views (see docstring)

In [ ]:
# Take a screenshot and display using matplotlib
# Note: this may fail with a shader-related error if you simply clicked "Run All" in a Jupyter notebook
# I think that's because we need to give it a split second to initialize (compile shaders, render a first frame, etc.)
# If you run this cell by itself, it should work fine
im = viewer.screenshot(filename=None)
im.shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(im)